In [1]:
import gym_2048
import gym
import numpy as np
import random
import copy

from itertools import permutations
from collections import Counter

In [2]:
def push_up(bb):
    s = 0
    s1 = 0

    b = copy.deepcopy(bb)
    b_new = copy.deepcopy(bb)

    for i in [0, 1, 2, 3]:
        l = []
        for j in range(0, 4):
            l.append(b[j][i])

        l = list(filter((0).__ne__, l))

        l_n = []
        while len(l) > 1:
            if l[0] == l[1] and l[0] != 0:
                l_n.append(2 * l[0])
                s = s + (2 * l[0])
                s1 = s1 + 1
                l.pop(0)
                l.pop(0)

            else:
                l_n.append(l[0])
                l.pop(0)
        if len(l) == 1:
            l_n.append(l[0])
            l.pop(0)

        for k in range(4 - len(l_n)):
            l_n.append(0)

        for j in range(0, 4):
            b_new[j][i] = l_n[j]
    return b_new, s, s1 * 16


def push_down(bb):
    s = 0
    s1 = 0

    b = copy.deepcopy(bb)
    b_new = copy.deepcopy(bb)

    for i in [0, 1, 2, 3]:
        l = []
        for j in [3, 2, 1, 0]:
            l.append(b[j][i])

        l = list(filter((0).__ne__, l))

        l_n = []
        while len(l) > 1:
            if l[0] == l[1] and l[0] != 0:
                l_n.append(2 * l[0])
                s = s + (2 * l[0])
                s1 = s1 + 1
                l.pop(0)
                l.pop(0)

            else:
                l_n.append(l[0])
                l.pop(0)
        if len(l) == 1:
            l_n.append(l[0])
            l.pop(0)

        for k in range(4 - len(l_n)):
            l_n.append(0)

        for j in range(0, 4):
            b_new[j][i] = l_n[3 - j]
    return b_new, s, s1


def push_left(bb):
    s = 0
    s1 = 0

    b = copy.deepcopy(bb)
    b_new = copy.deepcopy(bb)

    for i in [0, 1, 2, 3]:
        l = []
        for j in [0, 1, 2, 3]:
            l.append(b[i][j])

        l = list(filter((0).__ne__, l))

        l_n = []
        while len(l) > 1:
            if l[0] == l[1] and l[0] != 0:
                l_n.append(2 * l[0])
                s = s + (2 * l[0])
                s1 = s1 + 1
                l.pop(0)
                l.pop(0)

            else:
                l_n.append(l[0])
                l.pop(0)
        if len(l) == 1:
            l_n.append(l[0])
            l.pop(0)

        for k in range(4 - len(l_n)):
            l_n.append(0)

        for j in range(0, 4):
            b_new[i][j] = l_n[j]

    return b_new, s, s1


def push_right(bb):
    s = 0
    s1 = 0

    b = copy.deepcopy(bb)
    b_new = copy.deepcopy(bb)
    for i in [0, 1, 2, 3]:
        l = []
        for j in [3, 2, 1, 0]:
            l.append(b[i][j])

        l = list(filter((0).__ne__, l))

        l_n = []
        while len(l) > 1:
            if l[0] == l[1] and l[0] != 0:
                l_n.append(2 * l[0])
                s = s + (2 * l[0])
                s1 = s1 + 1
                l.pop(0)
                l.pop(0)
            else:
                l_n.append(l[0])
                l.pop(0)
        if len(l) == 1:
            l_n.append(l[0])
            l.pop(0)

        for k in range(4 - len(l_n)):
            l_n.append(0)

        for j in range(0, 4):
            b_new[i][j] = l_n[3 - j]

    return b_new, s, s1


def is_done(bb):
    b = copy.deepcopy(bb)

    if b == push_down(b)[0] and b == push_up(b)[0] and b == push_left(b)[0] and b == push_right(b)[0]:
        return True
    else:
        return False


def is_monotonic(L):
    return all(x >= y for x, y in zip(L, L[1:])) or all(x <= y for x, y in zip(L, L[1:]))

def get_total_merges(bb):
    b = copy.deepcopy(bb)
    count = push_down(b)[2] + push_left(b)[2]
    # print("Merges : ",push_down(b)[2]+push_left(b)[2])
    return count



def generate_monotonic_data():
    a = [True, False, True, False, True, False, True, False]

    b = list(permutations(a, 4))

    c = list(set(b))

    d = list(permutations(c + c, 2))

    return list(set(d))


def get_monotonicity_index(a):
    l = []
    for MN in [a, a.T]:
        k = []
        for i in MN:
            k.append(is_monotonic(i))

        l.append(tuple(k))

    return Monotonicity_master_data.index(tuple(l)), Counter(l)[True], Counter(l)[False]


def get_empty_cells(bb):
    count = 0
    b = copy.deepcopy(bb)

    for i in range(4):
        for j in range(4):
            if b[i][j] == 0:
                count = count + 1
    return count

In [3]:
Monotonicity_master_data = generate_monotonic_data()

env = gym.make('2048-v0')
# env.seed(42)

Q = np.zeros((256, 16, 9, 4))

env.reset()
env.render()

done = False

moves = 0
episodes = 100

explore_rate = 0.02
discount_factor = 0.8
learning_rate = 0.8

total_reward = 0

state = (0, 0, 0)

dump = []

0 	0 	0 	2
0 	0 	0 	2
0 	0 	0 	0
0 	0 	0 	0


In [4]:
for iteration in range(1, episodes + 1):
    
    prev_state = np.array([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]])
    
    env.reset()

    # print("Iteration : ", iteration)

    counter = 0

    while True:

        # print("")
        counter += 1

        # env.render()

        p = 0
        p = random.random()

        if p < explore_rate:
            action = env.action_space.sample()
        else:
            count = 0
            for i in (Q[state[0], state[1], state[2]]):
                if i == 0.0:
                    count = count + 1
            if count == 4:
                action = random.randint(0, 3)
            else:
                action = np.argmax(Q[state[0], state[1], state[2]])
            pass
        
        observation, reward, done, _ = env.step(action + 1)
        # print("Action : ", action + 1)

        merges = get_total_merges(observation)
        empty_cells = get_empty_cells(observation)
        monotonicity = get_monotonicity_index(np.array(observation))

#         print(reward)
        reward = merges + 10 * empty_cells + 10 * monotonicity[1] - 5 * monotonicity[2]

        act=[]
        if action == 0:
            act = [1,0,0,0]
        elif action==1:
            act = [0,1,0,0]
        elif action==2:
            act = [0,0,1,0]
        else:
            act = [0,0,0,1]

        p = []

        for m in prev_state:
            p += list(m)
            
        dump.append((p, reward, act))
        
        prev_state = observation

#         print(reward)
        total_reward = total_reward + reward

        q_alpha = np.max(Q[monotonicity[0], empty_cells, merges])

        Q[state][action] += learning_rate * (reward + discount_factor * q_alpha - Q[state][action])

        state = (monotonicity[0], empty_cells, merges)

        if done:
            print("\nIteration : ", iteration)
            print(observation)
            print("Unique Q:", (np.count_nonzero(Q)))
            break


Iteration :  1
[[ 2  8  4  2]
 [16  4 64  8]
 [ 8 16  2 16]
 [ 2  4 32  4]]
Unique Q: 86

Iteration :  2
[[  4   2  16   4]
 [  2  16  64   8]
 [ 64   8 128   2]
 [  4  64   2   8]]
Unique Q: 208

Iteration :  3
[[  2   4  16   4]
 [128   8  32  16]
 [ 16  64  16   4]
 [  4   2  32   2]]
Unique Q: 330

Iteration :  4
[[ 2  4  8  2]
 [ 4  8 32  4]
 [16  2 64 16]
 [ 2  4  8  4]]
Unique Q: 386

Iteration :  5
[[ 4  8  4  2]
 [32  4  2  4]
 [ 2 64 32  8]
 [ 4 32  4  2]]
Unique Q: 447

Iteration :  6
[[ 2  8  4  2]
 [ 8 16 64  8]
 [ 4  8 16 64]
 [ 2  4  8  2]]
Unique Q: 530

Iteration :  7
[[  4   2  16   4]
 [ 16 128  64   8]
 [  4   8  16   2]
 [  2  64   4   8]]
Unique Q: 617

Iteration :  8
[[ 2  8  2  4]
 [16 64  8 16]
 [ 4 16 64  4]
 [ 2  4  8  2]]
Unique Q: 666

Iteration :  9
[[ 2 16  8  2]
 [ 8 64 16  4]
 [ 2 16  8  2]
 [ 4  2 16  8]]
Unique Q: 714

Iteration :  10
[[ 4  8  4  8]
 [ 2 32  8  4]
 [ 4 64 32  2]
 [ 2  8 16  4]]
Unique Q: 772

Iteration :  11
[[ 4 16  8  4]
 [16 64 16


Iteration :  83
[[ 2  4  8 16]
 [ 8 32 16  2]
 [ 2  8 64  4]
 [ 8 16  4  2]]
Unique Q: 3459

Iteration :  84
[[ 8  2  4  2]
 [ 4 64 16  4]
 [32  2 32  8]
 [ 4 16  8  2]]
Unique Q: 3482

Iteration :  85
[[  2  16   8   2]
 [  4   8 256   8]
 [ 32   4  32   2]
 [  8   2   8   4]]
Unique Q: 3531

Iteration :  86
[[ 8 16  4  2]
 [ 4  8 64 16]
 [ 2 32  4  8]
 [ 4 16  8  2]]
Unique Q: 3550

Iteration :  87
[[  2  32  16   4]
 [ 32   2  32  64]
 [  4 128   4  16]
 [  2   4  32   2]]
Unique Q: 3583

Iteration :  88
[[  2   8   4   8]
 [  4 256  16   4]
 [  8  64   8   2]
 [  4   8   2   4]]
Unique Q: 3614

Iteration :  89
[[16  2  4  2]
 [ 4 64 16  8]
 [ 8 32  4  2]
 [ 2  4  8  4]]
Unique Q: 3632

Iteration :  90
[[  2   4   2   8]
 [  8  32  64  32]
 [  2  16 128   8]
 [  4   8   4   2]]
Unique Q: 3656

Iteration :  91
[[  4   2   4   2]
 [  8   4   8 128]
 [  4   2 128  16]
 [  2  32   8   2]]
Unique Q: 3678

Iteration :  92
[[ 4  2  8  2]
 [ 2  8 16  4]
 [16 32  8 64]
 [ 2 16  4  2]]
Uniqu

In [5]:
# dump=[]

In [6]:
len(dump)

20446

In [7]:
dump

[([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 131, [1, 0, 0, 0]),
 ([2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 130, [0, 1, 0, 0]),
 ([2, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 131, [0, 1, 0, 0]),
 ([0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], 121, [0, 0, 1, 0]),
 ([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 4, 4], 121, [0, 0, 1, 0]),
 ([0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 4, 0, 4, 4], 111, [1, 0, 0, 0]),
 ([2, 0, 4, 4, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 110, [0, 1, 0, 0]),
 ([0, 0, 2, 8, 2, 0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0], 100, [0, 0, 1, 0]),
 ([0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 8, 2, 0, 4, 2], 91, [0, 0, 1, 0]),
 ([0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 8, 2, 2, 4, 2], 82, [1, 0, 0, 0]),
 ([2, 2, 2, 2, 0, 0, 4, 8, 4, 0, 0, 2, 0, 0, 0, 0], 70, [0, 0, 1, 0]),
 ([0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 8, 4, 2, 4, 2], 72, [0, 0, 1, 0]),
 ([0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 8, 4, 4, 4, 2], 80, [0, 1, 0, 0]),
 ([0, 0, 2, 0, 0, 0, 0, 2, 0, 2, 4, 8, 0, 4, 8, 2], 72, [0, 0, 1, 0])

In [8]:
import os
os.environ['KERAS_BACKEND'] = 'theano'

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import np_utils

# import tensorflow as tf
import numpy
import pandas as pd
import theano

Using Theano backend.


In [9]:
model = Sequential()
model.add(Dense(100, input_dim=16, activation='relu'))
model.add(Dense(100, activation='relu'))

model.add(Dense(10, activation='relu'))

model.add(Dense(4, activation='softmax'))
# model.add(Activation(tf.nn.softmax))

# Compiling model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               1700      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 44        
Total params: 12,854
Trainable params: 12,854
Non-trainable params: 0
_________________________________________________________________


In [11]:
def get_vectors(dump):
    d = np.array(dump)
    l=d[:,0]
    x=list(l)
    x = np.array(x)
    p = d[:,2]
    y = list(p)
    y = np.array(y)
    return x, y

In [12]:
x, y = get_vectors(dump)

In [13]:
x

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 2,  0,  2, ...,  0,  0,  0],
       [ 2,  0,  2, ...,  0,  0,  0],
       ...,
       [ 4,  2, 16, ...,  4,  8,  2],
       [ 4,  2, 16, ...,  4,  8,  2],
       [ 4,  2, 16, ...,  4,  8,  2]])

In [14]:
model.fit(x, y, epochs=100)

Epoch 1/100
20446/20446 [==============================] - 0s 24us/step - loss: 1.3858 - acc: 0.2873
Epoch 2/100
20446/20446 [==============================] - 1s 28us/step - loss: 1.3228 - acc: 0.3057
Epoch 3/100
20446/20446 [==============================] - 1s 28us/step - loss: 1.3032 - acc: 0.3204
Epoch 4/100
20446/20446 [==============================] - 1s 32us/step - loss: 1.3016 - acc: 0.3135
Epoch 5/100
20446/20446 [==============================] - 1s 32us/step - loss: 1.2971 - acc: 0.3184
Epoch 6/100
20446/20446 [==============================] - 1s 31us/step - loss: 1.2955 - acc: 0.3148
Epoch 7/100
20446/20446 [==============================] - 1s 32us/step - loss: 1.2917 - acc: 0.3183
Epoch 8/100
20446/20446 [==============================] - 1s 31us/step - loss: 1.2904 - acc: 0.3191
Epoch 9/100
20446/20446 [==============================] - 1s 29us/step - loss: 1.2881 - acc: 0.3190
Epoch 10/100
20446/20446 [==============================] - 1s 32us/step - loss: 1.2851 - a

20446/20446 [==============================] - 1s 37us/step - loss: 1.1686 - acc: 0.4192
Epoch 82/100
20446/20446 [==============================] - 1s 49us/step - loss: 1.1594 - acc: 0.4243
Epoch 83/100
20446/20446 [==============================] - 1s 35us/step - loss: 1.1604 - acc: 0.4234
Epoch 84/100
20446/20446 [==============================] - 1s 43us/step - loss: 1.1599 - acc: 0.4244
Epoch 85/100
20446/20446 [==============================] - 1s 37us/step - loss: 1.1591 - acc: 0.4260
Epoch 86/100
20446/20446 [==============================] - 1s 33us/step - loss: 1.1592 - acc: 0.4271
Epoch 87/100
20446/20446 [==============================] - 1s 29us/step - loss: 1.1555 - acc: 0.4286
Epoch 88/100
20446/20446 [==============================] - 1s 29us/step - loss: 1.1400 - acc: 0.4394
Epoch 89/100
20446/20446 [==============================] - 1s 29us/step - loss: 1.1396 - acc: 0.4386
Epoch 90/100
20446/20446 [==============================] - 1s 29us/step - loss: 1.1343 - acc: 

In [15]:
# y_dash = model.predict(x.reshape(len(x),16))

In [16]:
# import keras.utils

In [17]:
# y_dash

In [18]:
# np.argmax(y_dash, axis=1)

In [19]:
explore_rate = 0.05

for iteration in range(10):

    env.reset()

    action = env.action_space.sample()
    observation, reward, done, _ = env.step(action + 1)


    while True:
#         env.render()

        counter += 1

        rand_action = env.action_space.sample()

        max_action = np.argmax(model.predict(np.ravel(np.array(list(observation))).reshape(1,16)), axis=1)[0]

        action = int(np.random.choice([rand_action, max_action], 1, p=[explore_rate, 1 - explore_rate]))

#         print("\nAction",action)

        observation, reward, done, _ = env.step(action + 1)

        if done:
            print("\nIteration : ", iteration)
            print(observation)
            break


Iteration :  0
[[ 4  2  4  2]
 [ 2  8 16  4]
 [ 4 16 64 16]
 [ 8 32  2 32]]

Iteration :  1
[[  2   8   2   4]
 [  4  16   4   2]
 [  8 128  16   8]
 [ 32  16   4  16]]

Iteration :  2
[[  2   4   2   4]
 [  4   8   4   8]
 [ 16 128  64  16]
 [ 32   2   4  64]]

Iteration :  3
[[  2   4   2   8]
 [  4   8   4  16]
 [ 16  64  16  32]
 [128  16 128   2]]

Iteration :  4
[[ 2  4  2  4]
 [ 4  8  4  2]
 [16  2 16  8]
 [ 2  4 32 16]]

Iteration :  5
[[  4   2   4   2]
 [  2  32   8   4]
 [ 32  64  32  16]
 [ 64 128   4  32]]

Iteration :  6
[[ 4  2  4  2]
 [ 8  4  2  8]
 [16  2  8 16]
 [32  4 16  2]]

Iteration :  7
[[  2   8   4   2]
 [  4  16   8   4]
 [  8  64  32   8]
 [ 32 128  64  32]]

Iteration :  8
[[ 4  2  8  4]
 [16  4 16  8]
 [ 2  8 32 16]
 [ 4 32 64  2]]

Iteration :  9
[[ 2  4  2  4]
 [ 4  8  4  2]
 [ 8 32  8  4]
 [32 64 32  8]]
